## For Demo

In [ ]:
# cols=['b1', 'b2', 'b3', 'b4', 'b5', 'b6', 
#          'green_0', 'green_2p', 'green_3p', 'green_4p', 'green_5p', 'green_6', 
#          'red_0', 'red_2p', 'red_3p', 'red_4p', 'red_5p', 'red_6', 
#          'blue_0', 'blue_2p', 'blue_3p', 'blue_4p', 'blue_5p', 'blue_6', 
#          'colours', 'total']
# df = pd.DataFrame(record_list, columns=cols)

# # Save the DataFrame as a CSV file
# df.to_csv('colour_records/rec_10.csv', index=False)

In [ ]:
import re

data = """
Capital = 100000
Profit Target = 150
Martingale Levels - [54, 73, 99, 135, 183, 248, 337, 457, 620, 842, 1143, 1551, 2105, 2857, 3877, 5262, 7141, 9691, 13152, 17849, 24224, 32875, 44617, 60551, 82177]
Martingale Sum = 312120
--------------------------------------------------
Initial Balance = 100000
25 -- (27, 23, 25) --> (7, 1, 0)
25 -- (27, 23, 25) --> (8, 2, 1)
25 -- (26, 24, 25) --> (9, 0, 2)
24 -- (26, 25, 25) --> (10, 0, 3)
25 -- (26, 25, 24) --> (11, 1, 4)
25 -- (26, 25, 24) --> (12, 0, 5)
24 -- (27, 25, 24) --> (0, 1, 6)
25 -- (27, 24, 24) --> (1, 2, 7)
25 -- (26, 25, 24) --> (2, 0, 8)
25 -- (26, 24, 25) --> (3, 1, 0)
26 -- (26, 23, 25) --> (4, 2, 1)
26 -- (26, 22, 26) --> (5, 3, 0)
26 -- (26, 22, 26) --> (6, 4, 0)
25 -- (27, 22, 26) --> (0, 5, 1)
26 -- (27, 21, 26) --> (1, 6, 2)
26 -- (28, 21, 25) --> (0, 7, 3) --->> TRADE
-------------------------------> Bet Placed Successfully ...
------------------------------->-------------------------------> green WIN = 1 Loss_levels = 0 Max Loss = 0
26 -- (29, 21, 24) --> (0, 8, 4)
"""

# tuples = []
# lines = data.split('\n')

# for line in lines:
#     match = re.search(r'(\d+) -- \((.*?)\) --> \((.*?)\)', line)
#     if match:
#         tuple_data = (int(match.group(1)),) + tuple(map(int, match.group(2).split(', '))) + tuple(map(int, match.group(3).split(', ')))
#         tuples.append(tuple_data)

# # print(tuples)
# record_list = tuples
# cols = ['none_T100', 'green_T100', 'red_T100', 'blue_T100', 'green_LD', 'red_LD', 'blue_LD']
# df = pd.DataFrame(record_list, columns=cols)
# # df['colours'] = df['colours'].astype(str)

# # Save the DataFrame as a CSV file
# df.to_csv('colour_records/rec_46_new.csv', index=False)

# Best Demo so far

In [1]:
import pandas as pd
import pywhatkit
import math
import datetime
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
import time
import sys


# # Set Chrome options to run the browser in the background
# chrome_options = Options()
# chrome_options.add_argument("--headless")  # Run Chrome in headless mode

service = Service("chrome_driver/chromedriver.exe")
driver = webdriver.Chrome(service=service)

# Create an instance of ActionChains
actions = ActionChains(driver)

# Open the specified URL
driver.get("https://logigames.bet9ja.com/Games/Launcher?gameId=11000&provider=0&pff=1&skin=201")
# driver.maximize_window()
time.sleep(5)

# Initialize an empty list to store records
record_list = []
start_trade = False
pause_trade = False
end_trade = False
capital = 100_000   # <<-------------------------------------------------------------------
print(f"Capital = {capital}")
win_count_target = 100 # Win times <<-------------------------------------------------------
win_count = 0
max_loss = []
loss_count = 0
max_num = 8
min_num = 6
last_draw_num = 13
green_count, red_count, blue_count = [], [], []
colour_list = [green_count, red_count, blue_count]
green_last_draw, red_last_draw, blue_last_draw = [], [], []
last_draw_list = [green_last_draw, red_last_draw, blue_last_draw]
draw_counter = 0
loss_count_list = []

# MARTINGALE LIST GENERATOR --------
target_percentage = 0.15  # %  <<-------------------------------------------------------------------
if target_percentage >= 1.32:
    print(f"Target Percentage is too high - {target_percentage}")
    sys.exit()
target = round(capital * (target_percentage / 100))
print(f"Profit Target = {target}")
odd = 3.8
martingale_levels = 25
martingale_loss = 0
martingale_stakes = []
for x in range(martingale_levels):
    stake = math.ceil((target + martingale_loss) / (odd - 1))
    martingale_stakes.append(stake)
    martingale_loss += stake
print(f"Martingale Levels = {martingale_stakes}")
print(f"Martingale Sum = {sum(martingale_stakes)}\n{'-'*50}")
Initial_balance = round(float(driver.find_element(By.XPATH, "//div[@class='rs-menu__balance-value']").text))
print(f"Initial Balance = {Initial_balance}")
# -------------------------------------------------------

while True:
    # Get the counter value from the webpage
    counter = driver.find_element(By.XPATH, "//div[@class='timeline__value-txt']").text

    draw_error = False
    if int(counter) < 41 and int(counter) > 37:
        draw_counter += 1
        trade_status = ""
        # Click on the statistics button
        try:
            driver.find_element(By.XPATH, "//div[@class='stats__toggle ']").click()     # -error check 1
        except Exception as e:  # handle any exception error
            print("---> Draw or Network error <---")
            draw_error = True
            driver.refresh()
            time.sleep(2)
            driver.find_element(By.XPATH, "//div[@class='stats__toggle ']").click()

        time.sleep(0.5)
        # Click on the "Rainbow" option in the dropdown menu
        driver.find_element(By.XPATH, "//a[contains(text(),'Total Colour')]").click()
        # Click on the "100 Draws" option in the dropdown menu
        driver.find_element(By.XPATH, "//div[@class='dd__trigger active dd__menu-open']//div[@class='dd__menu-item--value'][normalize-space()='100 Draws']").click()
        time.sleep(5)

        # Get the statistics text from the webpage
        stats = driver.find_element(By.XPATH, "//div[@class='p-stats__content']").text

        total_stats = tuple([int(x[-2:]) for x in stats.split("%")[:-1]])
        green_count.append(total_stats[1]), red_count.append(total_stats[2]), blue_count.append(total_stats[3])

        # This gets the Last Drawn values on the Statistics page
        last_drawn_record = tuple([int(x.split(" ")[0].split("\n")[1]) for x in stats.split("%")[1:]])
        green_last_draw.append(last_drawn_record[1]), red_last_draw.append(last_drawn_record[2]), blue_last_draw.append(last_drawn_record[3])
        if draw_counter > 1 and previous_stats_record == last_drawn_record:
            print("...... Draw correction ......")
            green_count.pop(), red_count.pop(), blue_count.pop()
            green_last_draw.pop(), red_last_draw.pop(), blue_last_draw.pop()
            driver.find_element(By.XPATH, "//a[normalize-space()='History']").click()  # Click on the "History" tab
            time.sleep(1.5)
            driver.find_element(By.XPATH, "//a[contains(text(),'Total Colour')]").click()  # Click on the "Total Colour" option in the dropdown menu
            time.sleep(0.5)
            driver.find_element(By.XPATH, "//div[@class='dd__trigger active dd__menu-open']//div[@class='dd__menu-item--value'][normalize-space()='100 Draws']").click()
            time.sleep(3)
            stats = driver.find_element(By.XPATH, "//div[@class='p-stats__content']").text
            total_stats = tuple([int(x[-2:]) for x in stats.split("%")[:-1]])
            green_count.append(total_stats[1]), red_count.append(total_stats[2]), blue_count.append(total_stats[3])
            last_drawn_record = tuple([int(x.split(" ")[0].split("\n")[1]) for x in stats.split("%")[1:]])
            green_last_draw.append(last_drawn_record[1]), red_last_draw.append(last_drawn_record[2]), blue_last_draw.append(last_drawn_record[3])
            if previous_stats_record == last_drawn_record:
                print(".... 2nd Draw correction ....")
                driver.refresh()
                time.sleep(1.5)
                green_count.pop(), red_count.pop(), blue_count.pop()
                green_last_draw.pop(), red_last_draw.pop(), blue_last_draw.pop()
                driver.find_element(By.XPATH, "//div[@class='stats__toggle ']").click()  # Click on stats
                time.sleep(1)
                driver.find_element(By.XPATH, "//a[contains(text(),'Total Colour')]").click()  # Click on the "Total Colour" option in the dropdown menu
                time.sleep(0.5)
                driver.find_element(By.XPATH, "//div[@class='dd__trigger active dd__menu-open']//div[@class='dd__menu-item--value'][normalize-space()='100 Draws']").click()
                time.sleep(2)
                stats = driver.find_element(By.XPATH, "//div[@class='p-stats__content']").text
                total_stats = tuple([int(x[-2:]) for x in stats.split("%")[:-1]])
                green_count.append(total_stats[1]), red_count.append(total_stats[2]), blue_count.append(total_stats[3])
                last_drawn_record = tuple([int(x.split(" ")[0].split("\n")[1]) for x in stats.split("%")[1:]])
                green_last_draw.append(last_drawn_record[1]), red_last_draw.append(last_drawn_record[2]), blue_last_draw.append(last_drawn_record[3])
                if (previous_stats_record == last_drawn_record) and draw_error == True:
                    time.sleep(5)
                    while True:
                        counter = driver.find_element(By.XPATH, "//div[@class='timeline__value-txt']").text
                        if int(counter) < 43 and int(counter) > 33:
                            print(".... 3rd Draw correction ....")
                            driver.refresh()
                            time.sleep(1.5)
                            green_count.pop(), red_count.pop(), blue_count.pop()
                            green_last_draw.pop(), red_last_draw.pop(), blue_last_draw.pop()
                            driver.find_element(By.XPATH, "//div[@class='stats__toggle ']").click()  # Click on stats
                            time.sleep(1)
                            driver.find_element(By.XPATH, "//a[contains(text(),'Total Colour')]").click()  # Click on the "Total Colour" option in the dropdown menu
                            time.sleep(0.5)
                            driver.find_element(By.XPATH, "//div[@class='dd__trigger active dd__menu-open']//div[@class='dd__menu-item--value'][normalize-space()='100 Draws']").click()
                            time.sleep(2)
                            stats = driver.find_element(By.XPATH, "//div[@class='p-stats__content']").text
                            total_stats = tuple([int(x[-2:]) for x in stats.split("%")[:-1]])
                            green_count.append(total_stats[1]), red_count.append(total_stats[2]), blue_count.append(total_stats[3])
                            last_drawn_record = tuple([int(x.split(" ")[0].split("\n")[1]) for x in stats.split("%")[1:]])
                            green_last_draw.append(last_drawn_record[1]), red_last_draw.append(last_drawn_record[2]), blue_last_draw.append(last_drawn_record[3])
                            break
                        else:
                            time.sleep(1)
        previous_stats_record = last_drawn_record

        # Check if result is a Win or Loss ----------------------------------------------------------------------------
        result_colours = [last_drawn_record[1], last_drawn_record[2], last_drawn_record[3]]
        if start_trade == True:
            if result_colours[colour_index] == 0:
                start_trade = False
                win_count += 1
                max_loss.append(loss_count)
                loss_count = 0
                # print(f"Total Wins = {win_count} --> Selected Colour = {colour_string} --> Last Loss count = {max_loss[-1]} --> Max Loss = {max(max_loss)}")
                print(f"------------------------------->-------------------------------> {colour_string} WIN = {win_count} Loss_levels = {max_loss[-1]} Max Loss = {max(max_loss)}")

            else:
                loss_count += 1
        # --------------------------------------------------------------------------------------------------------------
   

        # Click on the "History" tab
        driver.find_element(By.XPATH, "//a[normalize-space()='History']").click()
        time.sleep(2)
        
        # Get the history text from the webpage
        history = driver.find_element(By.XPATH, "//div[@class='p-stats__content']").text
        
        # This gets the number of times each colour appears from RED, GREEN and BLUE respectively
        # E.g '213' means 2 reds, 1 green and 3 blues
        colours_count = history.split(" ")[9:][0].split("\n")[-1]

        # This gets the Total number count for all six balls
        # E.g 148 means total sum for the given balls - 22, 14, 4, 35, 26, 47 
        sum_colours = int(history.split(" ")[9:][1].split("\n")[0])

        # Get the numbers for all 6 balls
        ball_path = "/html[1]/body[1]/div[1]/div[1]/div[1]/div[1]/main[1]/div[4]/div[2]/table[1]/tbody[1]/tr[1]/td[2]/div[1]/span["
        ball_1 = int(driver.find_element(By.XPATH, f"{ball_path}1]").text)
        ball_2 = int(driver.find_element(By.XPATH, f"{ball_path}2]").text)
        ball_3 = int(driver.find_element(By.XPATH, f"{ball_path}3]").text)
        ball_4 = int(driver.find_element(By.XPATH, f"{ball_path}4]").text)
        ball_5 = int(driver.find_element(By.XPATH, f"{ball_path}5]").text)
        ball_6 = int(driver.find_element(By.XPATH, f"{ball_path}6]").text)
        
        # Record all collected results in a listed tuple
        all_balls = (ball_1, ball_2, ball_3, ball_4, ball_5, ball_6)
        all_record = all_balls + total_stats + last_drawn_record + (colours_count, sum_colours)
        record_list.append(all_record)
        # print(all_record)

        # To open trading page
        driver.find_element(By.XPATH, "//div[@class='stats__toggle active']").click()
        time.sleep(1)

        if loss_count >= martingale_levels:
            print(f"\n      *** SORRY ***\n---> {loss_count} Losses Reached")
            # email_notification("TRADING STOPPED \nMAXIMUM LOSS REACHED ...")
            final_balance = round(float(driver.find_element(By.XPATH, "//div[@class='rs-menu__balance-value']").text))
            print(f"Initial Balance = {Initial_balance}")
            print(f"Final Balance = {final_balance}")
            print(f"Lost Amount = {final_balance - Initial_balance}")
            break

        if start_trade == False and (win_count >= win_count_target):
            driver.refresh()  # Reload webpage to get updated final balance
            time.sleep(2)
            print(f"\n    *** !!! CONGRATULATIONS !!! ***\n        {win_count} Wins Target Achieved")
            final_balance = round(float(driver.find_element(By.XPATH, "//div[@class='rs-menu__balance-value']").text))
            print(f"Initial Balance = {Initial_balance}")
            print(f"End Balance = {final_balance}")
            print(f"Profit = {final_balance - Initial_balance}")
            break

        # START TRADING CONDITION ------------------------------------------------------------------------------------------------------------------------.
        current_time = datetime.datetime.now().time()  # breaks the program if wrong trading time -------------------------------.                        |
        if (current_time >= datetime.time(23, 45) or current_time < datetime.time(1, 25)) and start_trade == False:#             |                        |
            print(f"\n     *** WRONG TRADING TIME ***\n     {win_count} Wins Achieved - We Continue Tomorrow ...")#              |                        |
            break  # ------------------------------------------------------------------------------------------------------------'                        |

        colours = ("green", "red", "blue")
        temp = total_stats[1:]
        # mid_num = sorted(list(temp))[-2]
        # if (draw_counter >= last_draw_num) and (temp.count(mid_num) == 1) and ((mid_num - min(temp)) <= 6) and (start_trade == False):
        #     colour_index = temp.index(mid_num)
        #     colour_num = temp[colour_index]
        #     colour_string = colours[colour_index]
        #     if (max(colour_list[colour_index][-max_num:-2]) < colour_num) and ((last_draw_list[colour_index][-10:]).count(0) >= 3):
        #         trade_status = "--->> mid TRADE"
        #         start_trade = True
                
        if (draw_counter >= last_draw_num) and (temp.count(max(temp)) == 1) and ((max(temp) - (sorted(list(temp))[-2])) <= 3) and (start_trade == False):
            colour_index = temp.index(max(temp))
            colour_num = temp[colour_index]
            colour_string = colours[colour_index]
            if (max(colour_list[colour_index][-max_num:-min_num]) < colour_num) and (total_stats[0] < (colour_num + 10)) and total_stats[0] > 18\
                and (max(last_draw_list[colour_index][-17:]) <= 5) and ((last_draw_list[colour_index][-10:]).count(0) >= 4)\
                and len(set(colour_list[colour_index][-4:])) > 1:                
                trade_status = "--->> max TRADE"
                start_trade = True      
        print(f"{total_stats[0]} -- {temp} --> {last_drawn_record[1:]} {trade_status}")#                                                                  |
        #                                                                                                                                                 |
        # ------------------------------------------------------------------------------------------------------------------------------------------------'


        # START TRADING if trade condition is True -----------------------------------------------------------------------------------------------------------
        if start_trade:
            # print("START TRADE HERE ...")
            before_balance = round(float(driver.find_element(By.XPATH, "//div[@class='rs-menu__balance-value']").text))
            driver.find_element(By.XPATH, "//a[5]").click()  # Total Colour
            time.sleep(1)
            
            stake_amount = martingale_stakes[loss_count]
            driver.find_element(By.XPATH, f"//div[@class='g-total__btn {colour_string} ']").click()  # Click on selected colour
            time.sleep(0.5)

            stake_box = driver.find_element(By.XPATH, "//body[1]/div[1]/div[1]/div[1]/div[1]/main[1]/div[2]/div[2]/div[3]/div[1]/div[2]/div[1]/div[1]/input[1]")
            stake_box.click()  # Click on stake box
            stake_box.send_keys(Keys.CONTROL + "a")  # Highlight the existing value in the stake box
            stake_box.send_keys(Keys.BACKSPACE)  # Clear the highlighted value
            time.sleep(0.5)
            
            stake_box.send_keys(stake_amount)  # Enter stake amount
            time.sleep(2)
            try:
                driver.find_element(By.XPATH, "//a[@class='place-bet']").click()  # Place bet
            
            # if there was a problem with PLACE BET button -------------------------------------------------------------------------------------------
            except ElementClickInterceptedException:
                print("Place bet button delay ---------------------------------->>>")
                driver.refresh()  # Reload the current webpage
                time.sleep(1)
                driver.find_element(By.XPATH, "//a[5]").click()  # Total Colour
                driver.find_element(By.XPATH, f"//div[@class='g-total__btn {colour_string} ']").click()
                time.sleep(1)

                stake_box = driver.find_element(By.XPATH, "//body[1]/div[1]/div[1]/div[1]/div[1]/main[1]/div[2]/div[2]/div[3]/div[1]/div[2]/div[1]/div[1]/input[1]")
                stake_box.click()  # Click on stake box
                stake_box.send_keys(Keys.CONTROL + "a")  # Highlight the existing value in the stake box
                stake_box.send_keys(Keys.BACKSPACE)  # Clear the highlighted value
                # time.sleep(0.5)
                
                stake_box.send_keys(stake_amount)  # Enter stake amount
                time.sleep(2)
                driver.find_element(By.XPATH, "//a[@class='place-bet']").click()  # Place bet
            time.sleep(2)
            driver.find_element(By.XPATH, f"//div[@class='g-total__btn {colour_string} active']").click()  # to unselect the last picked colour
            time.sleep(8)
            after_balance = round(float(driver.find_element(By.XPATH, "//div[@class='rs-menu__balance-value']").text))
            if before_balance != after_balance:
                # stake_status = True
                print("success -------------------->")
            else:
                print("!!! stake error !!!")
                driver.find_element(By.XPATH, f"//div[@class='g-total__btn {colour_string} ']").click()
                time.sleep(2)
                driver.find_element(By.XPATH, "//a[@class='place-bet']").click()  # Place bet
                time.sleep(5)
                after_balance = round(float(driver.find_element(By.XPATH, "//div[@class='rs-menu__balance-value']").text))
                if before_balance != after_balance:
                    # stake_status = True
                    print("stake success --------------------> 2")
                driver.find_element(By.XPATH, f"//div[@class='g-total__btn {colour_string} active']").click()  # to unselect
            # -----------------------------------------------------------------------------------------------------------------------------------------

    else: 
        time.sleep(1)  # 1 seconds counter delay

print(f"\nTotal Draws = {draw_counter}")
total_minutes = (draw_counter * 50) // 60
print(f"Total Wins = {win_count}")
print(f"Total Loss = {loss_count}")
print(f"Time Taken = {total_minutes // 60}hrs {total_minutes % 60}mins -----------------------------------------------")

# Close the browser session
driver.quit()

Capital = 100000
Profit Target = 150
Martingale Levels = [54, 73, 99, 135, 183, 248, 337, 457, 620, 842, 1143, 1551, 2105, 2857, 3877, 5262, 7141, 9691, 13152, 17849, 24224, 32875, 44617, 60551, 82177]
Martingale Sum = 312120
--------------------------------------------------
Initial Balance = 100000

     *** WRONG TRADING TIME ***
     0 Wins Achieved - We Continue Tomorrow ...

Total Draws = 1
Total Wins = 0
Total Loss = 0
Time Taken = 0hrs 0mins -----------------------------------------------


In [ ]:
# cols=['b1', 'b2', 'b3', 'b4', 'b5', 'b6', 
#          'green_0', 'green_2p', 'green_3p', 'green_4p', 'green_5p', 'green_6', 
#          'red_0', 'red_2p', 'red_3p', 'red_4p', 'red_5p', 'red_6', 
#          'blue_0', 'blue_2p', 'blue_3p', 'blue_4p', 'blue_5p', 'blue_6', 
#          'colours', 'total']

cols = ['b1', 'b2', 'b3', 'b4', 'b5', 'b6', 
      'none_T100', 'green_T100', 'red_T100', 'blue_T100', 
      'none_LD', 'green_LD', 'red_LD', 'blue_LD', 'colours', 'total']
df = pd.DataFrame(record_list, columns=cols)
df['colours'] = df['colours'].astype(str)

# Save the DataFrame as a CSV file
df.to_csv('colour_records/rec_67_new.csv', index=False)

# New Strategy test

In [1]:
import pandas as pd
import math
import datetime
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
import time
import sys

# =============================================================================================================
# service = Service("chrome_driver/chromedriver.exe")
# driver = webdriver.Chrome(service=service)

# driver.get("https://casino.bet9ja.com/casino/category/popular")
# # driver.maximize_window()
# time.sleep(5)

# # Create an instance of ActionChains
# actions = ActionChains(driver)

# # login 
# username = "Nnabueze"
# password = "cjval1610"
# print("Logging in ...")
# driver.find_element(By.XPATH, "//div[@title='Log In']").click()
# time.sleep(1)
# driver.find_element(By.XPATH, "//input[@id='01']").send_keys(username)
# driver.find_element(By.XPATH, "//input[@id='02']").send_keys(password)
# driver.find_element(By.XPATH, "//div[@class='btn-primary-l mt20']").click()
# time.sleep(3)

# element_to_hover = driver.find_element(By.XPATH, "//div[@id='11000']//div[@class='game__info']")
# actions.move_to_element(element_to_hover).perform()

# # # for real trading
# # driver.find_element(By.XPATH, "//div[@id='11000']//div[@class='game__info']//button[@title='Play Now'][normalize-space()='Play Now']").click()

# # for demo trading
# driver.find_element(By.XPATH, "//div[@id='11000']//button[@title='Demo'][normalize-space()='Demo']").click()

# time.sleep(5)

# # Get the handles of all open windows
# window_handles = driver.window_handles

# # Switch to the new window
# new_window_handle = window_handles[-1]  # Get the handle of the last window in the list
# driver.switch_to.window(new_window_handle)
# # print(len(window_handles))

# # Now you are in the new window

# # # Continue with actions or assertions in the new window
# # driver.maximize_window()
# time.sleep(1)
# =============================================================================================================
service = Service("chrome_driver/chromedriver.exe")
driver = webdriver.Chrome(service=service)

# Create an instance of ActionChains
actions = ActionChains(driver)

# Open the specified URL
driver.get("https://logigames.bet9ja.com/Games/Launcher?gameId=11000&provider=0&pff=1&skin=201")
# driver.maximize_window()
time.sleep(5)
# =============================================================================================================

start_time = time.time()  # Record the start time 
max_running_hours = 300  # Set the maximum running hours <<-------------------------------------------------------------------
max_running_time = max_running_hours * 60 * 60

# Initialize an empty list to store records
record_list = []
start_trade = False
pause_trade = False
end_trade = False

capital = 120_000   # <<-------------------------------------------------------------------
print(f"Capital = {capital}")
win_count_target = 1000 # Win times <<-------------------------------------------------------
win_count = 0
max_loss = []
loss_count = 0
# max_num = 8
# min_num = 6
# last_draw_num = 13
# green_count, red_count, blue_count = [], [], []
# colour_list = [green_count, red_count, blue_count]
green_last_draw, red_last_draw, blue_last_draw = [], [], []
# last_draw_list = [green_last_draw, red_last_draw, blue_last_draw]
draw_counter = 0
loss_count_list = []

# MARTINGALE LIST GENERATOR --------
target_percentage = 1.31  # %  <<-------------------------------------------------------------------
if target_percentage >= 1.32:
    print(f"Target Percentage is too high - {target_percentage}")
    sys.exit()
target = round(capital * (target_percentage / 100))
print(f"Profit Target = {target}")
odd = 3.8
martingale_levels = 16
martingale_loss = 0
martingale_stakes = []
for x in range(martingale_levels):
    stake = math.ceil((target + martingale_loss) / (odd - 1))
    martingale_stakes.append(stake)
    martingale_loss += stake
print(f"Martingale Levels = {martingale_stakes}")
print(f"Martingale Sum = {sum(martingale_stakes)}\n{'-'*50}")
Initial_balance = round(float(driver.find_element(By.XPATH, "//div[@class='rs-menu__balance-value']").text))
print(f"Initial Balance = {Initial_balance}")
# -------------------------------------------------------

def reloadResult(click_string="//div[@class='stats__toggle ']"):
    # global last_drawn_record, green_2plus_ld, red_2plus_ld, blue_2plus_ld
    # global last_drawn_record
    green_last_draw.pop(), red_last_draw.pop(), blue_last_draw.pop()
    driver.find_element(By.XPATH, click_string).click()  # Click on stats
    time.sleep(1)
    driver.find_element(By.XPATH, "//a[contains(text(),'Rainbow')]").click()  # Click on the "Total Colour" option in the dropdown menu
    time.sleep(0.5)
    driver.find_element(By.XPATH, "//div[@class='dd__trigger active dd__menu-open']//div[@class='dd__menu-item--value'][normalize-space()='All time']").click()
    time.sleep(2)

    # This gets the Last Drawn values on the Statistics page
    green_last_draw.append(int(driver.find_element(By.XPATH, "//tbody/tr[2]/td[4]").text))
    red_last_draw.append(int(driver.find_element(By.XPATH, "//tbody/tr[8]/td[4]").text))
    blue_last_draw.append(int(driver.find_element(By.XPATH, "//tbody/tr[14]/td[4]").text))
    # last_drawn_record = [green_last_draw[-1], red_last_draw[-1], blue_last_draw[-1]]

    # to confirm draw result
    green_6_ld = driver.find_element(By.CSS_SELECTOR, "tbody tr:nth-child(6) td:nth-child(4)").text
    red_6_ld = driver.find_element(By.CSS_SELECTOR, "tbody tr:nth-child(12) td:nth-child(4)").text
    blue_6_ld = driver.find_element(By.CSS_SELECTOR, "tbody tr:nth-child(18) td:nth-child(4)").text
    result_check = f"{green_6_ld}-{red_6_ld}-{blue_6_ld}"
    return result_check

while True:
    # Get the counter value from the webpage
    counter = driver.find_element(By.XPATH, "//div[@class='timeline__value-txt']").text

    draw_error = False
    if int(counter) < 41 and int(counter) > 37:
        draw_counter += 1
        trade_status = ""
        # Click on the statistics button
        try:
            driver.find_element(By.XPATH, "//div[@class='stats__toggle ']").click()     # -error check 1
        except Exception as e:  # handle any exception error
            print("---> Draw or Network error <---")
            draw_error = True
            driver.refresh()
            time.sleep(2)
            driver.find_element(By.XPATH, "//div[@class='stats__toggle ']").click()

        time.sleep(0.5)
        # Click on the "Rainbow" option in the dropdown menu
        driver.find_element(By.XPATH, "//a[contains(text(),'Rainbow')]").click()
        # Click on the "All time" option in the dropdown menu
        driver.find_element(By.XPATH, "//div[@class='dd__trigger active dd__menu-open']//div[@class='dd__menu-item--value'][normalize-space()='All time']").click()
        time.sleep(5)

        # This gets the Last Drawn values on the Statistics page
        green_last_draw.append(int(driver.find_element(By.XPATH, "//tbody/tr[2]/td[4]").text))
        red_last_draw.append(int(driver.find_element(By.XPATH, "//tbody/tr[8]/td[4]").text))
        blue_last_draw.append(int(driver.find_element(By.XPATH, "//tbody/tr[14]/td[4]").text))
        # last_drawn_record = [green_last_draw[-1], red_last_draw[-1], blue_last_draw[-1]]

        # to confirm draw result
        green_6_ld = driver.find_element(By.CSS_SELECTOR, "tbody tr:nth-child(6) td:nth-child(4)").text
        red_6_ld = driver.find_element(By.CSS_SELECTOR, "tbody tr:nth-child(12) td:nth-child(4)").text
        blue_6_ld = driver.find_element(By.CSS_SELECTOR, "tbody tr:nth-child(18) td:nth-child(4)").text
        result_check = f"{green_6_ld}-{red_6_ld}-{blue_6_ld}"
        # print("Result check --> ", result_check)

        if draw_counter > 1 and previous_stats_record == result_check:
            print("...... Draw correction ......")
            result_check = reloadResult(click_string="//a[normalize-space()='History']")
            if previous_stats_record == result_check:
                print(".... 2nd Draw correction ....")
                driver.refresh()
                time.sleep(1.5)
                result_check = reloadResult()
                if (previous_stats_record == result_check) and draw_error == True:
                    time.sleep(5)
                    while True:
                        counter = driver.find_element(By.XPATH, "//div[@class='timeline__value-txt']").text
                        if int(counter) < 43 and int(counter) > 33:
                            print(".... 3rd Draw correction ....")
                            driver.refresh()
                            time.sleep(1.5)
                            result_check = reloadResult()
                            break
                        else:
                            time.sleep(1)
        previous_stats_record = result_check

        # Check if result is a Win or Loss ----------------------------------------------------------------------------
        # result_colours = [last_drawn_record[1], last_drawn_record[2], last_drawn_record[3]]
        result_colours = [green_last_draw[-1], red_last_draw[-1], blue_last_draw[-1]]
        # print("new result --> ", result_colours)
        if start_trade == True:
            if result_colours[colour_index] == 0:
                start_trade = False
                win_count += 1
                max_loss.append(loss_count)
                loss_count = 0
                print(f"------------------------------->-------------------------------> {colour_string} WIN = {win_count} Loss_levels = {max_loss[-1]} Max Loss = {max(max_loss)}")

            else:
                loss_count += 1
        # --------------------------------------------------------------------------------------------------------------
   

        # Click on the "History" tab
        driver.find_element(By.XPATH, "//a[normalize-space()='History']").click()
        time.sleep(2)
        
        # Get the history text from the webpage
        history = driver.find_element(By.XPATH, "//div[@class='p-stats__content']").text
        
        # This gets the number of times each colour appears from RED, GREEN and BLUE respectively
        # E.g '213' means 2 reds, 1 green and 3 blues
        colours_count = history.split(" ")[9:][0].split("\n")[-1]

        # This gets the Total number count for all six balls
        # E.g 148 means total sum for the given balls - 22, 14, 4, 35, 26, 47 
        sum_colours = int(history.split(" ")[9:][1].split("\n")[0])

        # Get the numbers for all 6 balls
        ball_path = "/html[1]/body[1]/div[1]/div[1]/div[1]/div[1]/main[1]/div[4]/div[2]/table[1]/tbody[1]/tr[1]/td[2]/div[1]/span["
        ball_1 = int(driver.find_element(By.XPATH, f"{ball_path}1]").text)
        ball_2 = int(driver.find_element(By.XPATH, f"{ball_path}2]").text)
        ball_3 = int(driver.find_element(By.XPATH, f"{ball_path}3]").text)
        ball_4 = int(driver.find_element(By.XPATH, f"{ball_path}4]").text)
        ball_5 = int(driver.find_element(By.XPATH, f"{ball_path}5]").text)
        ball_6 = int(driver.find_element(By.XPATH, f"{ball_path}6]").text)
        
        last_5_colours = [num[-3:] for num in history.split(" ")[9:][0:-1:2][:5]]
        last_5_green, last_5_red, last_5_blue = [], [], []
        for numbers in last_5_colours:
            last_5_green.append(int(numbers[1]))
            last_5_red.append(int(numbers[0]))
            last_5_blue.append(int(numbers[2]))
        # print(last_5_red, last_5_green, last_5_blue)

        # Record all collected results in a listed tuple
        all_balls = (ball_1, ball_2, ball_3, ball_4, ball_5, ball_6)
        all_record = all_balls + (colours_count, sum_colours)
        record_list.append(all_record)
        # print(all_record)

        # To open trading page
        driver.find_element(By.XPATH, "//div[@class='stats__toggle active']").click()
        time.sleep(1)

        # if loss_count >= martingale_levels:
        #     print(f"\n      *** SORRY ***\n---> {loss_count} Losses Reached")
        #     # email_notification("TRADING STOPPED \nMAXIMUM LOSS REACHED ...")
        #     final_balance = round(float(driver.find_element(By.XPATH, "//div[@class='rs-menu__balance-value']").text))
        #     print(f"Initial Balance = {Initial_balance}")
        #     print(f"Final Balance = {final_balance}")
        #     print(f"Lost Amount = {final_balance - Initial_balance}")
        #     break

        if start_trade == False and (win_count >= win_count_target):
            driver.refresh()  # Reload webpage to get updated final balance
            time.sleep(2)
            print(f"\n    *** !!! CONGRATULATIONS !!! ***\n        {win_count} Wins Target Achieved")
            final_balance = round(float(driver.find_element(By.XPATH, "//div[@class='rs-menu__balance-value']").text))
            print(f"Initial Balance = {Initial_balance}")
            print(f"End Balance = {final_balance}")
            print(f"Profit = {final_balance - Initial_balance}")
            break

        if start_trade == False and ((time.time() - start_time) > max_running_time):  # Check if run time has exceeded the allocated trading time
            driver.refresh()  # Reload webpage to get updated final balance
            time.sleep(2)
            print(f"\n               *** ! PAUSE TRADING ! ***\n        {max_running_hours} hours Allocated Trading time exceeded")
            final_balance = round(float(driver.find_element(By.XPATH, "//div[@class='rs-menu__balance-value']").text))
            print(f"Initial Balance = {Initial_balance}")
            print(f"End Balance = {final_balance}")
            print(f"Profit = {final_balance - Initial_balance}")
            break

        # START TRADING CONDITION ------------------------------------------------------------------------------------------------------------------------.    
        colours = ("green", "red", "blue")#                                                                                                               |
        for pick in ["22442", "12424", "41042", "32231"]:  # "41042", "12424", "22442", "32231", "23432"
            n1, n2, n3, n4, n5 = map(int, pick)  # 41042, 22432, 32231, 12424
            if start_trade:
                break
            colour_index = -1
            for color in [last_5_green, last_5_red, last_5_blue]:
                colour_index += 1
                # print(f"{color[0]} - {n1}, {color[1]} - {n2}, {color[2]} - {n3}, {color[3]} - {n4}, {color[4]} - {n5}")
                if (color[0] >= n1) and (color[1] >= n2) and (color[2] >= n3) and (color[3] >= n4) and (color[4] >= n5):
                    start_trade = True
                    # print(colour_index)
                    colour_string = colours[colour_index]
                    trade_status = f"---> stake {colour_string.upper()} - {pick}"#                                                                              |
                    break
                # if notification_start == False:  # For sending email notification to start trading -----------------------.                             |
                #     email_notification("AUTO TRADING STARTED")#                                                           |                             |
                #     notification_start = True  # -------------------------------------------------------------------------'                             |
        #                                                                                                                                                 |
        last_5 = f'[{"".join(map(str, last_5_red))} {"".join(map(str, last_5_green))} {"".join(map(str, last_5_blue))}]'
        print(f"R.G.B - {colours_count}-{last_5}-{red_last_draw[-1], green_last_draw[-1], blue_last_draw[-1]} {trade_status}")#              |
        #                                                                                                                                                 |
        # ------------------------------------------------------------------------------------------------------------------------------------------------'


        # START TRADING if trade condition is True --------------------------------------------------------------------------------------------------------
        if start_trade:
            green_string = "//div[contains(@class,'game__ts')]//div[3]//div[2]//div[1]//div[1]"
            red_string = "//div[contains(@class,'rainbow-grid')]//div[1]//div[2]//div[1]//div[1]"
            blue_string = "//div[contains(@class,'game__content')]//div[2]//div[2]//div[1]//div[1]"
            colors_string_list = [green_string, red_string, blue_string]
            color_select = colors_string_list[colour_index]
            color_unselect = f"div[class='rainbow__ball {colour_string} available active'] div[class='rainbow__ball-value']"
            # color_unselect = f"//div[@class='rainbow__ball red available active']//div[@class='rainbow__ball-value'][normalize-space()='2+']"

            # print("START TRADE HERE ...")
            before_balance = round(float(driver.find_element(By.XPATH, "//div[@class='rs-menu__balance-value']").text))
            driver.find_element(By.XPATH, "//a[4]").click()  # Total Colour
            time.sleep(1)

            stake_amount = martingale_stakes[loss_count]
            # Click on selected colour
            driver.find_element(By.XPATH, color_select).click()
            time.sleep(0.5)

            stake_box = driver.find_element(By.XPATH, "//body[1]/div[1]/div[1]/div[1]/div[1]/main[1]/div[2]/div[2]/div[3]/div[1]/div[2]/div[1]/div[1]/input[1]")
            stake_box.click()  # Click on stake box
            stake_box.send_keys(Keys.CONTROL + "a")  # Highlight the existing value in the stake box
            stake_box.send_keys(Keys.BACKSPACE)  # Clear the highlighted value
            time.sleep(0.5)
            
            stake_box.send_keys(stake_amount)  # Enter stake amount
            time.sleep(2)
            try:
                driver.find_element(By.XPATH, "//a[@class='place-bet']").click()  # Place bet
            
            # if there was a problem with PLACE BET button -------------------------------------------------------------------------------------------
            except ElementClickInterceptedException:
                print("Place bet button delay ---------------------------------->>>")
                driver.refresh()  # Reload the current webpage
                time.sleep(1)
                driver.find_element(By.XPATH, "//a[4]").click()  # Total Colour
                driver.find_element(By.XPATH, color_select).click()
                time.sleep(1)

                stake_box = driver.find_element(By.XPATH, "//body[1]/div[1]/div[1]/div[1]/div[1]/main[1]/div[2]/div[2]/div[3]/div[1]/div[2]/div[1]/div[1]/input[1]")
                stake_box.click()  # Click on stake box
                stake_box.send_keys(Keys.CONTROL + "a")  # Highlight the existing value in the stake box
                stake_box.send_keys(Keys.BACKSPACE)  # Clear the highlighted value
                # time.sleep(0.5)
                
                stake_box.send_keys(stake_amount)  # Enter stake amount
                time.sleep(2)
                driver.find_element(By.XPATH, "//a[@class='place-bet']").click()  # Place bet
            time.sleep(2)
            driver.find_element(By.CSS_SELECTOR, color_unselect).click()  # to unselect the last picked colour
            time.sleep(8)
            after_balance = round(float(driver.find_element(By.XPATH, "//div[@class='rs-menu__balance-value']").text))
            if before_balance != after_balance:
                # stake_status = True
                print("success -------------------->")
            else:
                print("!!! stake error !!!")
                driver.find_element(By.XPATH, color_select).click()
                time.sleep(2)
                driver.find_element(By.XPATH, "//a[@class='place-bet']").click()  # Place bet
                time.sleep(5)
                after_balance = round(float(driver.find_element(By.XPATH, "//div[@class='rs-menu__balance-value']").text))
                if before_balance != after_balance:
                    # stake_status = True
                    print("stake success --------------------> 2")
                driver.find_element(By.CSS_SELECTOR, color_unselect).click()  # to unselect
            # -----------------------------------------------------------------------------------------------------------------------------------------

    else: 
        time.sleep(1)  # 1 seconds counter delay

print(f"\nTotal Draws = {draw_counter}")
total_minutes = (draw_counter * 50) // 60
print(f"Total Wins = {win_count}")
print(f"Total Loss = {loss_count}")
print(f"Time Taken = {total_minutes // 60}hrs {total_minutes % 60}mins -----------------------------------------------")

# Close the browser session
driver.quit()

Capital = 120000
Profit Target = 1572
Martingale Levels = [562, 763, 1035, 1405, 1907, 2588, 3512, 4766, 6468, 8778, 11913, 16168, 21942, 29779, 40414, 54848]
Martingale Sum = 206848
--------------------------------------------------
Initial Balance = 100000
R.G.B - 213-[20321 11214 34131]-(0, 2, 0) 
R.G.B - 141-[12032 41121 13413]-(1, 0, 1) 
R.G.B - 114-[11203 14112 41341]-(2, 1, 0) 
R.G.B - 141-[11120 41411 14134]-(3, 0, 1) 
R.G.B - 321-[31112 24141 11413]-(0, 0, 2) 
R.G.B - 221-[23111 22414 11141]-(0, 0, 3) 
R.G.B - 231-[22311 32241 11114]-(0, 0, 4) ---> stake GREEN - 32231
success -------------------->
R.G.B - 303-[32231 03224 31111]-(0, 1, 0) 
success -------------------->
------------------------------->-------------------------------> green WIN = 1 Loss_levels = 1 Max Loss = 1
R.G.B - 041-[03223 40322 13111]-(1, 0, 1) 
R.G.B - 213-[20322 14032 31311]-(0, 1, 0) 
R.G.B - 141-[12032 41403 13131]-(1, 0, 1) 
R.G.B - 213-[21203 14140 31313]-(0, 1, 0) 
R.G.B - 114-[12120 11414 43131]-(

WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=128.0.6613.138)
Stacktrace:
	GetHandleVerifier [0x00007FF619949412+29090]
	(No symbol) [0x00007FF6198BE239]
	(No symbol) [0x00007FF61977B1DA]
	(No symbol) [0x00007FF6197628FC]
	(No symbol) [0x00007FF6197627C0]
	(No symbol) [0x00007FF61977DAC1]
	(No symbol) [0x00007FF619815ED9]
	(No symbol) [0x00007FF6197F6493]
	(No symbol) [0x00007FF6197C09D1]
	(No symbol) [0x00007FF6197C1B31]
	GetHandleVerifier [0x00007FF619C6871D+3302573]
	GetHandleVerifier [0x00007FF619CB4243+3612627]
	GetHandleVerifier [0x00007FF619CAA417+3572135]
	GetHandleVerifier [0x00007FF619A05EB6+801862]
	(No symbol) [0x00007FF6198C945F]
	(No symbol) [0x00007FF6198C4FB4]
	(No symbol) [0x00007FF6198C5140]
	(No symbol) [0x00007FF6198B461F]
	BaseThreadInitThunk [0x00007FFB2EF8257D+29]
	RtlUserThreadStart [0x00007FFB3050AF28+40]


In [6]:
# cols=['b1', 'b2', 'b3', 'b4', 'b5', 'b6', 
#          'green_0', 'green_2p', 'green_3p', 'green_4p', 'green_5p', 'green_6', 
#          'red_0', 'red_2p', 'red_3p', 'red_4p', 'red_5p', 'red_6', 
#          'blue_0', 'blue_2p', 'blue_3p', 'blue_4p', 'blue_5p', 'blue_6', 
#          'colours', 'total']

cols = ['b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'colours', 'total']
df = pd.DataFrame(record_list, columns=cols)
df['colours'] = df['colours'].astype(str)

# Save the DataFrame as a CSV file
df.to_csv('colour_records/rec_108_new.csv', index=False)

In [ ]:
"""
06 07 08 09 10 11 12 13 14 15
16 17 18 19 20 21 22 23 24 25
42 43 44 45 46 47 48 49 50 51

499s per 10 draws ----> 49.9s per draw ---> 479s
"""

In [ ]:
"""
Capital = 120000
Profit Target = 1572
Martingale Levels = [562, 763, 1035, 1405, 1907, 2588, 3512, 4766, 6468, 8778, 11913, 16168, 21942, 29779, 40414, 54848]
Martingale Sum = 206848
--------------------------------------------------
Initial Balance = 100000
R.G.B - 222 --> ['23014', '20312', '23340'] - LD(0, 0, 0) 
R.G.B - 213 --> ['22301', '12031', '32334'] - LD(0, 1, 0) ---> stake BLUE - 32231
success -------------------->
------------------------------->-------------------------------> blue WIN = 1 Loss_levels = 0 Max Loss = 0
R.G.B - 114 --> ['12230', '11203', '43233'] - LD(1, 2, 0) ---> stake BLUE - 32231
success -------------------->
R.G.B - 420 --> ['41223', '21120', '04323'] - LD(0, 0, 1) 
success -------------------->
------------------------------->-------------------------------> blue WIN = 2 Loss_levels = 1 Max Loss = 1
R.G.B - 213 --> ['24122', '12112', '30432'] - LD(0, 1, 0) 
R.G.B - 212 --> ['22412', '11211', '23043'] - LD(0, 2, 0) 
R.G.B - 222 --> ['22241', '21121', '22304'] - LD(0, 0, 0) 
R.G.B - 132 --> ['12224', '32112', '22230'] - LD(1, 0, 0) 
R.G.B - 240 --> ['21222', '43211', '02223'] - LD(0, 0, 1) 
R.G.B - 042 --> ['02122', '44321', '20222'] - LD(1, 0, 0) 
R.G.B - 402 --> ['40212', '04432', '22022'] - LD(0, 1, 0) 
R.G.B - 033 --> ['04021', '30443', '32202'] - LD(1, 0, 0) 
R.G.B - 123 --> ['10402', '23044', '33220'] - LD(2, 0, 0) 
R.G.B - 123 --> ['11040', '22304', '33322'] - LD(3, 0, 0) 
R.G.B - 213 --> ['21104', '12230', '33332'] - LD(0, 1, 0) ---> stake BLUE - 32231
success -------------------->
------------------------------->-------------------------------> blue WIN = 3 Loss_levels = 0 Max Loss = 1
R.G.B - 032 --> ['02110', '31223', '23333'] - LD(1, 0, 0) 
R.G.B - 104 --> ['10211', '03122', '42333'] - LD(2, 1, 0) ---> stake BLUE - 32231
success -------------------->
------------------------------->-------------------------------> blue WIN = 4 Loss_levels = 0 Max Loss = 1
R.G.B - 222 --> ['21021', '20312', '24233'] - LD(0, 0, 0) 
R.G.B - 132 --> ['12102', '32031', '22423'] - LD(1, 0, 0) 
R.G.B - 051 --> ['01210', '53203', '12242'] - LD(2, 0, 1) 
R.G.B - 320 --> ['30121', '25320', '01224'] - LD(0, 0, 2) 
R.G.B - 132 --> ['13012', '32532', '20122'] - LD(1, 0, 0) ---> stake GREEN - 32231
success -------------------->
R.G.B - 213 --> ['21301', '13253', '32012'] - LD(0, 1, 0) 
success -------------------->
------------------------------->-------------------------------> green WIN = 5 Loss_levels = 1 Max Loss = 1
R.G.B - 123 --> ['12130', '21325', '33201'] - LD(1, 0, 0) 
R.G.B - 123 --> ['11213', '22132', '33320'] - LD(2, 0, 0) 
R.G.B - 320 --> ['31121', '22213', '03332'] - LD(0, 0, 1) 
R.G.B - 321 --> ['33112', '22221', '10333'] - LD(0, 0, 2) 
R.G.B - 231 --> ['23311', '32222', '11033'] - LD(0, 0, 3) 
R.G.B - 312 --> ['32331', '13222', '21103'] - LD(0, 1, 0) ---> stake RED - 32231
success -------------------->
R.G.B - 005 --> ['03233', '01322', '52110'] - LD(1, 2, 0) 
success -------------------->
R.G.B - 023 --> ['00323', '20132', '35211'] - LD(2, 0, 0) 
success -------------------->
------------------------------->-------------------------------> red WIN = 6 Loss_levels = 2 Max Loss = 2
R.G.B - 222 --> ['20032', '22013', '23521'] - LD(0, 0, 0) 
R.G.B - 132 --> ['12003', '32201', '22352'] - LD(1, 0, 0) 
R.G.B - 222 --> ['21200', '23220', '22235'] - LD(0, 0, 0) 
R.G.B - 231 --> ['22120', '32322', '12223'] - LD(0, 0, 1) 
R.G.B - 132 --> ['12212', '33232', '21222'] - LD(1, 0, 0) ---> stake GREEN - 32231
success -------------------->
------------------------------->-------------------------------> green WIN = 7 Loss_levels = 0 Max Loss = 2
R.G.B - 123 --> ['11221', '23323', '32122'] - LD(2, 0, 0) 
R.G.B - 132 --> ['11122', '32332', '23212'] - LD(3, 0, 0) ---> stake GREEN - 32231
success -------------------->
------------------------------->-------------------------------> green WIN = 8 Loss_levels = 0 Max Loss = 2
R.G.B - 051 --> ['01112', '53233', '12321'] - LD(4, 0, 1) ---> stake GREEN - 32231
success -------------------->
------------------------------->-------------------------------> green WIN = 9 Loss_levels = 0 Max Loss = 2
R.G.B - 321 --> ['30111', '25323', '11232'] - LD(0, 0, 2) 
R.G.B - 131 --> ['13011', '32532', '11123'] - LD(1, 0, 3) ---> stake GREEN - 32231
success -------------------->
R.G.B - 312 --> ['31301', '13253', '21112'] - LD(0, 1, 0) 
success -------------------->
------------------------------->-------------------------------> green WIN = 10 Loss_levels = 1 Max Loss = 2
R.G.B - 222 --> ['23130', '21325', '22111'] - LD(0, 0, 0) 
R.G.B - 330 --> ['32313', '32132', '02211'] - LD(0, 0, 1) 
R.G.B - 123 --> ['13231', '23213', '30221'] - LD(1, 0, 0) 
R.G.B - 123 --> ['11323', '22321', '33022'] - LD(2, 0, 0) 
R.G.B - 123 --> ['11132', '22232', '33302'] - LD(3, 0, 0) 
R.G.B - 141 --> ['11113', '42223', '13330'] - LD(4, 0, 1) 
R.G.B - 204 --> ['21111', '04222', '41333'] - LD(0, 1, 0) 
R.G.B - 222 --> ['22111', '20422', '24133'] - LD(0, 0, 0) 
R.G.B - 213 --> ['22211', '12042', '32413'] - LD(0, 1, 0) 
R.G.B - 024 --> ['02221', '21204', '43241'] - LD(1, 0, 0) ---> stake BLUE - 32231
success -------------------->
------------------------------->-------------------------------> blue WIN = 11 Loss_levels = 0 Max Loss = 2
R.G.B - 222 --> ['20222', '22120', '24324'] - LD(0, 0, 0) 
R.G.B - 321 --> ['32022', '22212', '12432'] - LD(0, 0, 1) 
R.G.B - 114 --> ['13202', '12221', '41243'] - LD(1, 1, 0) ---> stake BLUE - 41042
success -------------------->
------------------------------->-------------------------------> blue WIN = 12 Loss_levels = 0 Max Loss = 2
R.G.B - 312 --> ['31320', '11222', '24124'] - LD(0, 2, 0) 
R.G.B - 411 --> ['43132', '11122', '12412'] - LD(0, 3, 1) 
R.G.B - 230 --> ['24313', '31112', '01241'] - LD(0, 0, 2) 
R.G.B - 213 --> ['22431', '13111', '30124'] - LD(0, 1, 0) 
R.G.B - 321 --> ['32243', '21311', '13012'] - LD(0, 0, 1) ---> stake RED - 32231
success -------------------->
------------------------------->-------------------------------> red WIN = 13 Loss_levels = 0 Max Loss = 2
R.G.B - 213 --> ['23224', '12131', '31301'] - LD(0, 1, 0) 
R.G.B - 123 --> ['12322', '21213', '33130'] - LD(1, 0, 0) 
R.G.B - 213 --> ['21232', '12121', '33313'] - LD(0, 1, 0) 
R.G.B - 213 --> ['22123', '11212', '33331'] - LD(0, 2, 0) ---> stake BLUE - 32231
success -------------------->
------------------------------->-------------------------------> blue WIN = 14 Loss_levels = 0 Max Loss = 2
R.G.B - 132 --> ['12212', '31121', '23333'] - LD(1, 0, 0) 
R.G.B - 222 --> ['21221', '23112', '22333'] - LD(0, 0, 0) 
R.G.B - 303 --> ['32122', '02311', '32233'] - LD(0, 1, 0) ---> stake BLUE - 32231
success -------------------->
------------------------------->-------------------------------> blue WIN = 15 Loss_levels = 0 Max Loss = 2
R.G.B - 212 --> ['23212', '10231', '23223'] - LD(0, 2, 0) 
R.G.B - 222 --> ['22321', '21023', '22322'] - LD(0, 0, 0) 
R.G.B - 141 --> ['12232', '42102', '12232'] - LD(1, 0, 1) 
R.G.B - 231 --> ['21223', '34210', '11223'] - LD(0, 0, 2) 
R.G.B - 321 --> ['32122', '23421', '11122'] - LD(0, 0, 3) 
R.G.B - 221 --> ['23212', '22342', '11112'] - LD(0, 0, 4) 
R.G.B - 321 --> ['32321', '22234', '11111'] - LD(0, 0, 5) 
R.G.B - 114 --> ['13232', '12223', '41111'] - LD(1, 1, 0) 
R.G.B - 132 --> ['11323', '31222', '24111'] - LD(2, 0, 0) 
R.G.B - 123 --> ['11132', '23122', '32411'] - LD(3, 0, 0) 
R.G.B - 033 --> ['01113', '32312', '33241'] - LD(4, 0, 0) ---> stake BLUE - 32231
success -------------------->
------------------------------->-------------------------------> blue WIN = 16 Loss_levels = 0 Max Loss = 2
R.G.B - 132 --> ['10111', '33231', '23324'] - LD(5, 0, 0) ---> stake GREEN - 32231
success -------------------->
------------------------------->-------------------------------> green WIN = 17 Loss_levels = 0 Max Loss = 2
R.G.B - 051 --> ['01011', '53323', '12332'] - LD(6, 0, 1) 
R.G.B - 231 --> ['20101', '35332', '11233'] - LD(0, 0, 2) ---> stake GREEN - 32231
success -------------------->
------------------------------->-------------------------------> green WIN = 18 Loss_levels = 0 Max Loss = 2
R.G.B - 024 --> ['02010', '23533', '41123'] - LD(1, 0, 0) 
R.G.B - 221 --> ['20201', '22353', '14112'] - LD(0, 0, 1) 
R.G.B - 033 --> ['02020', '32235', '31411'] - LD(1, 0, 0) ---> stake GREEN - 32231
success -------------------->
------------------------------->-------------------------------> green WIN = 19 Loss_levels = 0 Max Loss = 2
R.G.B - 321 --> ['30202', '23223', '13141'] - LD(0, 0, 1) 
R.G.B - 231 --> ['23020', '32322', '11314'] - LD(0, 0, 2) 
R.G.B - 222 --> ['22302', '23232', '21131'] - LD(0, 0, 0) 
R.G.B - 240 --> ['22230', '42323', '02113'] - LD(0, 0, 1) 
R.G.B - 303 --> ['32223', '04232', '30211'] - LD(0, 1, 0) 
R.G.B - 213 --> ['23222', '10423', '33021'] - LD(0, 2, 0) 
R.G.B - 060 --> ['02322', '61042', '03302'] - LD(1, 0, 1) ---> stake GREEN - 41042
success -------------------->
------------------------------->-------------------------------> green WIN = 20 Loss_levels = 0 Max Loss = 2
R.G.B - 123 --> ['10232', '26104', '30330'] - LD(2, 0, 0) 
R.G.B - 222 --> ['21023', '22610', '23033'] - LD(0, 0, 0) 
R.G.B - 231 --> ['22102', '32261', '12303'] - LD(0, 0, 1) ---> stake GREEN - 32231
success -------------------->
------------------------------->-------------------------------> green WIN = 21 Loss_levels = 0 Max Loss = 2
R.G.B - 122 --> ['12210', '23226', '21230'] - LD(1, 0, 0) 
R.G.B - 330 --> ['31221', '32322', '02123'] - LD(0, 0, 1) 
R.G.B - 321 --> ['33122', '23232', '10212'] - LD(0, 0, 2) 
R.G.B - 123 --> ['13312', '22323', '31021'] - LD(1, 0, 0) 
R.G.B - 312 --> ['31331', '12232', '23102'] - LD(0, 1, 0) 
R.G.B - 231 --> ['23133', '31223', '12310'] - LD(0, 0, 1) 
R.G.B - 024 --> ['02313', '23122', '41231'] - LD(1, 0, 0) 
R.G.B - 141 --> ['10231', '42312', '14123'] - LD(2, 0, 1) 
R.G.B - 420 --> ['41023', '24231', '01412'] - LD(0, 0, 2) 
R.G.B - 312 --> ['34102', '12423', '20141'] - LD(0, 1, 0) 
R.G.B - 213 --> ['23410', '11242', '32014'] - LD(0, 2, 0) 
R.G.B - 141 --> ['12341', '41124', '13201'] - LD(1, 0, 1) 
R.G.B - 222 --> ['21234', '24112', '21320'] - LD(0, 0, 0) 
R.G.B - 321 --> ['32123', '22411', '12132'] - LD(0, 0, 1) 
R.G.B - 303 --> ['33212', '02241', '31213'] - LD(0, 1, 0) 
R.G.B - 141 --> ['13321', '40224', '13121'] - LD(1, 0, 1) 
R.G.B - 222 --> ['21332', '24022', '21312'] - LD(0, 0, 0) 
R.G.B - 114 --> ['12133', '12402', '42131'] - LD(1, 1, 0) 
R.G.B - 231 --> ['21213', '31240', '14213'] - LD(0, 0, 1) 
R.G.B - 123 --> ['12121', '23124', '31421'] - LD(1, 0, 0) 
R.G.B - 312 --> ['31212', '12312', '23142'] - LD(0, 1, 0) 
R.G.B - 222 --> ['23121', '21231', '22314'] - LD(0, 0, 0) 
R.G.B - 212 --> ['22312', '12123', '22231'] - LD(0, 1, 0) 
R.G.B - 222 --> ['22231', '21212', '22223'] - LD(0, 0, 0) 
R.G.B - 231 --> ['22223', '32121', '12222'] - LD(0, 0, 1) 
R.G.B - 321 --> ['32222', '23212', '11222'] - LD(0, 0, 2) 
R.G.B - 212 --> ['23222', '12321', '21122'] - LD(0, 1, 0) 
R.G.B - 213 --> ['22322', '11232', '32112'] - LD(0, 2, 0) 
R.G.B - 141 --> ['12232', '41123', '13211'] - LD(1, 0, 1) 
R.G.B - 222 --> ['21223', '24112', '21321'] - LD(0, 0, 0) 
R.G.B - 231 --> ['22122', '32411', '12132'] - LD(0, 0, 1) 
R.G.B - 213 --> ['22212', '13241', '31213'] - LD(0, 1, 0) 
R.G.B - 321 --> ['32221', '21324', '13121'] - LD(0, 0, 1) 
R.G.B - 240 --> ['23222', '42132', '01312'] - LD(0, 0, 2) 
R.G.B - 221 --> ['22322', '24213', '10131'] - LD(0, 0, 3) 
R.G.B - 321 --> ['32232', '22421', '11013'] - LD(0, 0, 4) ---> stake RED - 32231
success -------------------->
------------------------------->-------------------------------> red WIN = 22 Loss_levels = 0 Max Loss = 2
R.G.B - 213 --> ['23223', '12242', '31101'] - LD(0, 1, 0) 
R.G.B - 213 --> ['22322', '11224', '33110'] - LD(0, 2, 0) 
R.G.B - 213 --> ['22232', '11122', '33311'] - LD(0, 3, 0) 
R.G.B - 222 --> ['22223', '21112', '23331'] - LD(0, 0, 0) 
R.G.B - 114 --> ['12222', '12111', '42333'] - LD(1, 1, 0) ---> stake BLUE - 32231
success -------------------->
R.G.B - 231 --> ['21222', '31211', '14233'] - LD(0, 0, 1) 
success -------------------->
------------------------------->-------------------------------> blue WIN = 23 Loss_levels = 1 Max Loss = 2
R.G.B - 132 --> ['12122', '33121', '21423'] - LD(1, 0, 0) 
R.G.B - 330 --> ['31212', '33312', '02142'] - LD(0, 0, 1) 
R.G.B - 330 --> ['33121', '33331', '00214'] - LD(0, 0, 2) ---> stake GREEN - 32231
success -------------------->
------------------------------->-------------------------------> green WIN = 24 Loss_levels = 0 Max Loss = 2
R.G.B - 032 --> ['03312', '33333', '20021'] - LD(1, 0, 0) ---> stake GREEN - 32231
success -------------------->
------------------------------->-------------------------------> green WIN = 25 Loss_levels = 0 Max Loss = 2
R.G.B - 222 --> ['20331', '23333', '22002'] - LD(0, 0, 0) 
R.G.B - 411 --> ['42033', '12333', '12200'] - LD(0, 1, 1) 
R.G.B - 114 --> ['14203', '11233', '41220'] - LD(1, 2, 0) 
R.G.B - 420 --> ['41420', '21123', '04122'] - LD(0, 0, 1) 
R.G.B - 420 --> ['44142', '22112', '00412'] - LD(0, 0, 2) ---> stake RED - 41042
success -------------------->
------------------------------->-------------------------------> red WIN = 26 Loss_levels = 0 Max Loss = 2
R.G.B - 231 --> ['24414', '32211', '10041'] - LD(0, 0, 3) 
R.G.B - 231 --> ['22441', '33221', '11004'] - LD(0, 0, 4) 
R.G.B - 132 --> ['12244', '33322', '21100'] - LD(1, 0, 0) 
R.G.B - 113 --> ['11224', '13332', '32110'] - LD(2, 1, 0) 
R.G.B - 213 --> ['21122', '11333', '33211'] - LD(0, 2, 0) 
R.G.B - 141 --> ['12112', '41133', '13321'] - LD(1, 0, 1) 
R.G.B - 222 --> ['21211', '24113', '21332'] - LD(0, 0, 0) 
R.G.B - 123 --> ['12121', '22411', '32133'] - LD(1, 0, 0) 
R.G.B - 321 --> ['31212', '22241', '13213'] - LD(0, 0, 1) 
R.G.B - 411 --> ['43121', '12224', '11321'] - LD(0, 1, 2) 
R.G.B - 132 --> ['14312', '31222', '21132'] - LD(1, 0, 0) 
R.G.B - 132 --> ['11431', '33122', '22113'] - LD(2, 0, 0) 
R.G.B - 213 --> ['21143', '13312', '32211'] - LD(0, 1, 0) 
R.G.B - 033 --> ['02114', '31331', '33221'] - LD(1, 0, 0) 
R.G.B - 132 --> ['10211', '33133', '23322'] - LD(2, 0, 0) 
R.G.B - 123 --> ['11021', '23313', '32332'] - LD(3, 0, 0) ---> stake BLUE - 32231
success -------------------->
------------------------------->-------------------------------> blue WIN = 27 Loss_levels = 0 Max Loss = 2
R.G.B - 302 --> ['31102', '02331', '23233'] - LD(0, 1, 0) 
R.G.B - 042 --> ['03110', '40233', '22323'] - LD(1, 0, 0) 
R.G.B - 222 --> ['20311', '24023', '22232'] - LD(0, 0, 0) 
R.G.B - 222 --> ['22031', '22402', '22223'] - LD(0, 0, 0) 
R.G.B - 132 --> ['12203', '32240', '22222'] - LD(1, 0, 0) 
R.G.B - 312 --> ['31220', '13224', '22222'] - LD(0, 1, 0) 
R.G.B - 240 --> ['23122', '41322', '02222'] - LD(0, 0, 1) 
R.G.B - 321 --> ['32312', '24132', '10222'] - LD(0, 0, 2) 
R.G.B - 132 --> ['13231', '32413', '21022'] - LD(1, 0, 0) 
R.G.B - 122 --> ['11323', '23241', '22102'] - LD(2, 0, 0) 
R.G.B - 312 --> ['31132', '12324', '22210'] - LD(0, 1, 0) 
R.G.B - 420 --> ['43113', '21232', '02221'] - LD(0, 0, 1) 
R.G.B - 312 --> ['34311', '12123', '20222'] - LD(0, 1, 0) 
R.G.B - 132 --> ['13431', '31212', '22022'] - LD(1, 0, 0) 
R.G.B - 411 --> ['41343', '13121', '12202'] - LD(0, 1, 1) ---> stake RED - 41042
success -------------------->
R.G.B - 132 --> ['14134', '31312', '21220'] - LD(1, 0, 0) 
success -------------------->
------------------------------->-------------------------------> red WIN = 28 Loss_levels = 1 Max Loss = 2
R.G.B - 222 --> ['21413', '23131', '22122'] - LD(0, 0, 0) 
R.G.B - 122 --> ['12141', '22313', '22212'] - LD(1, 0, 0) 
R.G.B - 024 --> ['01214', '22231', '42221'] - LD(2, 0, 0) 
R.G.B - 303 --> ['30121', '02223', '34222'] - LD(0, 1, 0) 
R.G.B - 213 --> ['23012', '10222', '33422'] - LD(0, 2, 0) 
R.G.B - 141 --> ['12301', '41022', '13342'] - LD(1, 0, 1) 
R.G.B - 204 --> ['21230', '04102', '41334'] - LD(0, 1, 0) 
R.G.B - 402 --> ['42123', '00410', '24133'] - LD(0, 2, 0) 
R.G.B - 114 --> ['14212', '10041', '42413'] - LD(1, 3, 0) 
R.G.B - 123 --> ['11421', '21004', '34241'] - LD(2, 0, 0) ---> stake BLUE - 32231
success -------------------->
------------------------------->-------------------------------> blue WIN = 29 Loss_levels = 0 Max Loss = 2
R.G.B - 123 --> ['11142', '22100', '33424'] - LD(3, 0, 0) ---> stake BLUE - 12424
success -------------------->
R.G.B - 231 --> ['21114', '32210', '13342'] - LD(0, 0, 1) 
success -------------------->
------------------------------->-------------------------------> blue WIN = 30 Loss_levels = 1 Max Loss = 2
R.G.B - 222 --> ['22111', '23221', '21334'] - LD(0, 0, 0) 
R.G.B - 123 --> ['12211', '22322', '32133'] - LD(1, 0, 0) 
R.G.B - 231 --> ['21221', '32232', '13213'] - LD(0, 0, 1) ---> stake GREEN - 32231
success -------------------->
------------------------------->-------------------------------> green WIN = 31 Loss_levels = 0 Max Loss = 2
R.G.B - 321 --> ['32122', '23223', '11321'] - LD(0, 0, 2) 
R.G.B - 123 --> ['13212', '22322', '31132'] - LD(1, 0, 0) 
R.G.B - 240 --> ['21321', '42232', '03113'] - LD(0, 0, 1) ---> stake GREEN - 32231
success -------------------->
R.G.B - 213 --> ['22132', '14223', '30311'] - LD(0, 1, 0) 
success -------------------->
------------------------------->-------------------------------> green WIN = 32 Loss_levels = 1 Max Loss = 2
R.G.B - 122 --> ['12213', '21422', '23031'] - LD(1, 0, 0) 
R.G.B - 132 --> ['11221', '32142', '22303'] - LD(2, 0, 0) 
R.G.B - 032 --> ['01122', '33214', '22230'] - LD(3, 0, 0) 
R.G.B - 321 --> ['30112', '23321', '12223'] - LD(0, 0, 1) 
...... Draw correction ......

"""